In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [38]:
from selenium import webdriver

driver=webdriver.Chrome()

driver.get("https://catalog.onliner.by/robotcleaner")

In [58]:
# Собираем все ссылки на объявления с выбранной страницы
def get_all_links_on_page():
    elements = driver.find_elements(By.CLASS_NAME, value='catalog-form__link')
    # elements = driver.find_elements(by=By.CSS_SELECTOR, value="#container > div > div > div > div > div.catalog-content > div.catalog-wrapper > div > div > div.catalog-form__tabs > div > div > div > div > div.catalog-form__filter-part.catalog-form__filter-part_2 > div.catalog-form__offers > div > div:nth-child(1) > div > div > div.catalog-form__offers-part.catalog-form__offers-part_data > div.catalog-form__description.catalog-form__description_primary.catalog-form__description_base-additional.catalog-form__description_font-weight_semibold.catalog-form__description_condensed-other > a")
    # <a href="https://catalog.onliner.by/robotcleaner" class="catalog-masthead__title</a>
    ads_list = []
    for element in elements:
        ads_list.append(element.get_attribute('href'))
        # print(element.get_attribute('href'), '\n')

    return ads_list

In [59]:
links = get_all_links_on_page()

In [60]:
links

['https://tech.onliner.by/2025/07/15/kakoj-robot-pylesos-vybrat',
 None,
 None,
 'https://catalog.onliner.by/robotcleaner/xiaomi/5prog20/prices',
 None,
 None,
 'https://catalog.onliner.by/robotcleaner/xiaomi/5prog20',
 'https://forum.onliner.by/posting.php?mode=newtopic&f=164&device=5prog20',
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr089reu/prices',
 None,
 None,
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr089reu',
 'https://forum.onliner.by/posting.php?mode=newtopic&f=164&device=bhr089reu',
 'https://catalog.onliner.by/robotcleaner/dreame/x50ultracomplete/prices',
 None,
 None,
 'https://catalog.onliner.by/robotcleaner/dreame/x50ultracomplete',
 'https://forum.onliner.by/posting.php?mode=newtopic&f=164&device=x50ultracomplete',
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr0834eubhr7357/prices',
 None,
 None,
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr0834eubhr7357',
 'https://forum.onliner.by/posting.php?mode=newtopic&f=164&device=bhr0834eubhr7357',


In [56]:
# Переход на следующую страницу через нажатие на кнопку
def click_next_page():
    clickable_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="container"]/div/div/div/div/div[2]/div[1]/div/div/div[4]/div/div/div/div/div[3]/div[3]/a'))
    )

    clickable_element.click()

In [ ]:
click_next_page()

In [61]:
len(links)


157

In [62]:
clear_links = []

for item in links:
    if item == None or 'forum.' in item or 'tech.' in item:
        pass
    else:
        clear_links.append(item)

len(clear_links)

67

In [63]:
very_clear_links = []

for item in clear_links:
    if 'prices' in item or 'reviews' in item or 'used' in item:
        pass
    else:
        very_clear_links.append(item)

len(very_clear_links)

30

In [64]:
very_clear_links

['https://catalog.onliner.by/robotcleaner/xiaomi/5prog20',
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr089reu',
 'https://catalog.onliner.by/robotcleaner/dreame/x50ultracomplete',
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr0834eubhr7357',
 'https://catalog.onliner.by/robotcleaner/xiaomi/d102gl',
 'https://catalog.onliner.by/robotcleaner/dreame/rlx63ce2wh',
 'https://catalog.onliner.by/robotcleaner/dreame/rll74ce',
 'https://catalog.onliner.by/robotcleaner/dreame/rll22sewh',
 'https://catalog.onliner.by/robotcleaner/roborock/saros10rbl',
 'https://catalog.onliner.by/robotcleaner/roborock/qrevocurvwhru',
 'https://catalog.onliner.by/robotcleaner/deerma/demx70',
 'https://catalog.onliner.by/robotcleaner/deerma/dems30',
 'https://catalog.onliner.by/robotcleaner/dreame/f9prow',
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr8159eu',
 'https://catalog.onliner.by/robotcleaner/xiaomi/xiaomi5pro',
 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr9220eu',
 'https://catal

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

м

data = []

for url in very_clear_links:
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        title = soup.find("h1", class_="catalog-masthead__title")
        title = title.get_text(strip=True) if title else None
        
        price = soup.find("a", class_="offers-description__price")
        price = price.get_text(strip=True) if price else None
        
        characteristics = {}
        char_table = soup.find("div", class_="product-specs__table")
        if char_table:
            rows = char_table.find_all("div", class_="product-specs__row")
            for row in rows:
                key = row.find("div", class_="product-specs__title")
                value = row.find("div", class_="product-specs__value")
                if key and value:
                    characteristics[key.get_text(strip=True)] = value.get_text(strip=True)
        
        data.append({
            "url": url,
            "title": title,
            "price": price,
            "characteristics": characteristics
        })
        
        time.sleep(1)
    else:
        print(f"Ошибка при загрузке {url}")

df = pd.DataFrame(data)

print(df.head())


                                                 url  \
0  https://catalog.onliner.by/robotcleaner/xiaomi...   
1  https://catalog.onliner.by/robotcleaner/xiaomi...   
2  https://catalog.onliner.by/robotcleaner/dreame...   
3  https://catalog.onliner.by/robotcleaner/xiaomi...   
4  https://catalog.onliner.by/robotcleaner/xiaomi...   

                                               title price characteristics  
0  Робот-пылесос Xiaomi Robot Vacuum 5 Pro OV21GL...  None              {}  
1  Робот-пылесос Xiaomi Robot Vacuum S40 Pro OV71...  None              {}  
2  Робот-пылесос Dreame Robot Vacuum X50 Ultra Co...  None              {}  
3  Робот-пылесос Xiaomi Robot Vacuum 5 OV31GL (ев...  None              {}  
4  Робот-пылесос Xiaomi Robot Vacuum X20 Pro D102...  None              {}  


In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Запускаем браузер
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # можно убрать, если хочешь видеть браузер
driver = webdriver.Chrome(options=options)

# 1. Открываем каталог
driver.get("https://catalog.onliner.by/robotcleaner")
time.sleep(3)  # ждём прогрузки

# 2. Собираем ссылки на товары
elements = driver.find_elements(By.CLASS_NAME, "catalog-form__link")
links = [el.get_attribute("href") for el in elements if el.get_attribute("href")]

# фильтруем только карточки товаров
product_links = [
    link for link in links
    if link and all(x not in link for x in ["forum.", "tech.", "prices", "reviews", "used"])
]

print(f"Найдено {len(product_links)} ссылок")

# 3. Парсим карточки
data = []

for url in product_links:
    driver.get(url)
    time.sleep(2)  # ждём прогрузки JS

    # Название
    try:
        title = driver.find_element(By.CLASS_NAME, "catalog-masthead__title").text
    except:
        title = None

    # Цена
    try:
        price = driver.find_element(By.CLASS_NAME, "offers-description__price").text
    except:
        price = None

    # Характеристики
    characteristics = {}
    try:
        rows = driver.find_elements(By.CLASS_NAME, "product-specs__row")
        for row in rows:
            try:
                key = row.find_element(By.CLASS_NAME, "product-specs__title").text
                value = row.find_element(By.CLASS_NAME, "product-specs__value").text
                characteristics[key] = value
            except:
                pass
    except:
        pass

    data.append({
        "url": url,
        "title": title,
        "price": price,
        "characteristics": characteristics
    })

# Закрываем браузер
driver.quit()

# 4. В DataFrame
df = pd.DataFrame(data)
print(df.head())

# Сохраняем в Excel/CSV при необходимости
# df.to_excel("onliner_products.xlsx", index=False)
# df.to_csv("onliner_products.csv", index=False, encoding="utf-8-sig")

Найдено 6 ссылок
                                                 url  \
0  https://catalog.onliner.by/robotcleaner/xiaomi...   
1  https://catalog.onliner.by/robotcleaner/xiaomi...   
2  https://catalog.onliner.by/robotcleaner/dreame...   
3  https://catalog.onliner.by/robotcleaner/xiaomi...   
4  https://catalog.onliner.by/robotcleaner/xiaomi...   

                                               title          price  \
0  Робот-пылесос Xiaomi Robot Vacuum 5 Pro OV21GL...     3299,00 р.   
1  Робот-пылесос Xiaomi Robot Vacuum S40 Pro OV71...  от 1066,00 р.   
2  Робот-пылесос Dreame Robot Vacuum X50 Ultra Co...  от 3898,99 р.   
3  Робот-пылесос Xiaomi Robot Vacuum 5 OV31GL (ев...     2699,00 р.   
4  Робот-пылесос Xiaomi Robot Vacuum X20 Pro D102...  от 1719,00 р.   

  characteristics  
0              {}  
1              {}  
2              {}  
3              {}  
4              {}  


In [68]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Твои 30 ссылок (можно вставить прямо сюда или загрузить из CSV)
very_clear_links

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # убрать, если хочешь видеть браузер
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)

data = []

for url in very_clear_links:
    driver.get(url)
    time.sleep(2)  # ждём прогрузки JS

    # Название
    try:
        title = driver.find_element(By.CLASS_NAME, "catalog-masthead__title").text
    except:
        title = None

    # Цена
    try:
        price = driver.find_element(By.CLASS_NAME, "offers-description__price").text
    except:
        price = None

    # Характеристики
    characteristics = {}
    try:
        rows = driver.find_elements(By.CLASS_NAME, "product-specs__row")
        for row in rows:
            try:
                key = row.find_element(By.CLASS_NAME, "product-specs__title").text
                value = row.find_element(By.CLASS_NAME, "product-specs__value").text
                characteristics[key] = value
            except:
                pass
    except:
        pass

    data.append({
        "url": url,
        "title": title,
        "price": price,
        "characteristics": characteristics
    })

driver.quit()

# В DataFrame
df = pd.DataFrame(data)

# Разворачиваем характеристики в отдельные колонки
df_expanded = df.join(df["characteristics"].apply(pd.Series))
df_expanded.drop(columns=["characteristics"], inplace=True)

# Сохраняем в CSV
df_expanded.to_csv("onliner_products.csv", index=False, encoding="utf-8-sig")

print("Готово! Данные сохранены в onliner_products.csv")



Готово! Данные сохранены в onliner_products.csv


In [ ]:
data

[{'url': 'https://catalog.onliner.by/robotcleaner/xiaomi/5prog20',
  'title': 'Робот-пылесос Xiaomi Robot Vacuum 5 Pro OV21GL + Пылесос Xiaomi Vacuum Cleaner G20 D205 (евровилка, белый)',
  'price': '3299,00 р.',
  'characteristics': {}},
 {'url': 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr089reu',
  'title': 'Робот-пылесос Xiaomi Robot Vacuum S40 Pro OV71GL (евровилка, белый)',
  'price': 'от 1066,00 р.',
  'characteristics': {}},
 {'url': 'https://catalog.onliner.by/robotcleaner/dreame/x50ultracomplete',
  'title': 'Робот-пылесос Dreame Robot Vacuum X50 Ultra Complete RLX85CE-4 (евровилка, черный)',
  'price': 'от 3898,99 р.',
  'characteristics': {}},
 {'url': 'https://catalog.onliner.by/robotcleaner/xiaomi/bhr0834eubhr7357',
  'title': 'Робот-пылесос Xiaomi Robot Vacuum 5 OV31GL (евровилка, белый) + Аэрогриль Xiaomi Smart Air Fryer 6.5L MAF10 (евровилка, черный)',
  'price': '2699,00 р.',
  'characteristics': {}},
 {'url': 'https://catalog.onliner.by/robotcleaner/xiaomi/d10